# Profiling & Optimization in Python

## About me

<center>
    <img src='resources/about-me.png'>
</center>       

## Optimization

## 3 Rules of optimization

### Don't  💡

Always have measurable performance goals before optimizing code.

### Don't Yet 💡

Developer time is expensive, sometimes the best solution is to upgrade hardware.

### Profile before Optimizing 💡

Profiling measures where code spends time.

## Measuring time

### Two modules

- time
    - time.clock() obsoleto en 3.3
    - time.monotonic()
    - time.perf_counter()
    - time.process_time()
    - time.time()

- timeit

- https://www.webucator.com/blog/2015/08/python-clocks-explained/

#### Examples perf_counter:

In [2]:
def upto_for(n):
    total = 0
    for i in range(n):
        total += i
    return total

def upto_sum(n):
    return sum(range(n))

In [1]:
n = 1_000_000

In [4]:
from time import perf_counter

In [5]:
start = perf_counter()
upto_for(n)
duration = perf_counter() - start
print("upto_for", duration)

upto_for 0.07374520799999829


In [6]:
start = perf_counter()
upto_sum(n)
duration = perf_counter() - start
print("upto_sum", duration)

upto_sum 0.0348250830000012


#### Examples timeit:

In [7]:
from timeit import timeit

items = {
    'a': 1,
    'b': 2,
}

default = -1

def use_catch(key):
    """Use try/catch to get a key with default"""
    try:
        return items[key]
    except KeyError:
        return default

def use_get(key):
    """Use dict.get to get a key with default"""
    return items.get(key, default)

In [8]:
# Key is in the dictionary
print('catch', timeit('use_catch("x")', 'from __main__ import use_catch', number=5000000))
print('get', timeit('use_get("x")', 'from __main__ import use_get', number=5000000))

catch 0.7718442920000044
get 0.3403776250000021


### Magic function timeit:

In [9]:
%timeit use_catch("a")

54.8 ns ± 0.331 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [10]:
%timeit use_get("a")

72 ns ± 0.201 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


## CPU Profiling

### Deterministic profiling 
*record every function call and return, as well as exceptions*


### Statistical profiling
*record where the program is at small intervals*


## Profiling in Python


### Deterministic profiling 

- #### cProfile (C extension)
- #### profile (pure Python module)             


## cProfile

#### C extension

#### Based on lsprof

#### Deterministic profiling 


#### cProfile is built to trace a single thread
- multi-threaded application ([Yappi](https://github.com/yousong/yappi)).

## Example CProfile

In [12]:
from crypt import crypt
import sqlite3

salt = '$6$ZmBkxkRFj03LQOvr'  # Bad security, store safely
db = sqlite3.connect('./passwords.db')
db.row_factory = sqlite3.Row  # Access columns by names

In [13]:
def user_passwd(user):
    """Get user password from db"""
    cur = db.cursor()
    cur.execute('SELECT passwd FROM users WHERE user = ?', (user, ))
    row = cur.fetchone()
    if row is None:  # No such user
        raise KeyError(user)
    return row['passwd']


def encrypt_passwd(passwd):
    """Encrypt user password"""
    return crypt(passwd, salt)

In [14]:
def login(user, password):
    """Return True is user/password pair matches"""
    try:
        db_passwd = user_passwd(user)
    except KeyError:
        return False

    passwd = encrypt_passwd(password)
    return passwd == db_passwd

In [15]:
from random import random


def gen_cases(n):
    """Generate tests cases"""
    for i in range(n):
        if random() > 0.1:  # 90% of logins are OK
            yield ('daffy', 'rabbit season')
        else:
            if random() < 0.2:
                yield ('tweety', 'puddy tat')  # no such user
            else:
                yield ('daffy', 'duck season')


def bench_login(cases):
    """Benchmark login with test cases"""
    for user, passwd in cases:
        login(user, passwd)

In [16]:
n = 1000
cases = list(gen_cases(n))

In [17]:
import cProfile

cProfile.run('bench_login(cases)', sort="cumtime")

         8912 function calls in 0.036 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.036    0.036 {built-in method builtins.exec}
        1    0.000    0.000    0.036    0.036 <string>:1(<module>)
        1    0.000    0.000    0.036    0.036 2615380777.py:16(bench_login)
     1000    0.002    0.000    0.035    0.000 2293933993.py:1(login)
     1000    0.001    0.000    0.024    0.000 1035362234.py:1(user_passwd)
     1000    0.019    0.000    0.019    0.000 {method 'execute' of 'sqlite3.Cursor' objects}
      977    0.000    0.000    0.009    0.000 1035362234.py:11(encrypt_passwd)
      977    0.001    0.000    0.008    0.000 crypt.py:70(crypt)
      977    0.008    0.000    0.008    0.000 {built-in method _crypt.crypt}
     1000    0.003    0.000    0.003    0.000 {method 'fetchone' of 'sqlite3.Cursor' objects}
     1000    0.001    0.000    0.001    0.000 {method 'cursor' of 'sqlite3.C

In [18]:
cProfile.run('bench_login(cases)', filename="profiling.out")

In [19]:
import pstats
from pstats import SortKey

sortby = SortKey.CUMULATIVE

ps = pstats.Stats("profiling.out").strip_dirs().sort_stats(sortby)

In [20]:
ps.print_stats(10)

Fri Nov  5 09:58:43 2021    profiling.out

         8912 function calls in 0.040 seconds

   Ordered by: cumulative time
   List reduced from 13 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.040    0.040 {built-in method builtins.exec}
        1    0.000    0.000    0.040    0.040 <string>:1(<module>)
        1    0.000    0.000    0.040    0.040 2615380777.py:16(bench_login)
     1000    0.002    0.000    0.040    0.000 2293933993.py:1(login)
     1000    0.001    0.000    0.027    0.000 1035362234.py:1(user_passwd)
     1000    0.021    0.000    0.021    0.000 {method 'execute' of 'sqlite3.Cursor' objects}
      977    0.001    0.000    0.010    0.000 1035362234.py:11(encrypt_passwd)
      977    0.001    0.000    0.009    0.000 crypt.py:70(crypt)
      977    0.009    0.000    0.009    0.000 {built-in method _crypt.crypt}
     1000    0.004    0.000    0.004    0.000 {method 'fetchone' of 'sqli

## snakeviz

In [21]:
%load_ext snakeviz

In [22]:
%snakeviz bench_login(cases)

 
*** Profile stats marshalled to file '/var/folders/g5/_gc0xsms1t7dsqg9_ssm3_bdwn9xh3/T/tmpj7xu4u3l'. 
Embedding SnakeViz in this document...


## Example line_profiler

In [23]:
%load_ext line_profiler

In [24]:
%lprun -T lprof0 -f login bench_login(cases)


*** Profile printout saved to text file 'lprof0'. 


## Tracing Memory Allocation

- Programs contain data.
- They are stored in the memory. 
- Accessing memory becomes important.
- In modern computers it is done in layers.


### Latency

<center>
    <img src='resources/latency.png'>
</center>                    


## tracemalloc & memory_profiler

#### - The tracemalloc module is a debug tool to trace memory blocks allocated by Python. (New in 3.4)


#### - memory_profiler

- tracemalloc nos permite entender como esta asignando python la memoria en nuestro programa.
- memory profiler nos permite ver linea por linea donde se esta yendo la memoria.

## Example memory_profiler

In [25]:
%%file memory.py

import numpy as np

@profile
def allocate():
    vector_list = [float(i) for i in range(1_000_000)]
    vector_np = np.arange(0, 1_000_000, dtype="d")
    
allocate()

Writing memory.py


In [26]:
! python -m memory_profiler memory.py

Filename: memory.py

Line #    Mem usage    Increment  Occurences   Line Contents
     4   53.984 MiB   53.984 MiB           1   @profile
     5                                         def allocate():
     6   92.609 MiB   38.625 MiB     1000003       vector_list = [float(i) for i in range(1_000_000)]
     7  100.250 MiB    7.641 MiB           1       vector_np = np.arange(0, 1_000_000, dtype="d")




In [27]:
! mprof run memory.py

mprof: Sampling memory every 0.1s
running new process
running as a Python program...


In [33]:
! mprof plot -o image.png


Using last profile data.


<center>
    <img src='./image.png'>
</center>  

## In production

<center>
    <img src='resources/production.png'>
</center>                    

- https://docs.newrelic.com/docs/apm/apm-ui-pages/events/thread-profiler-tool
- https://docs.datadoghq.com/tracing/profiler/


<center>
    <img src='resources/example_profile_2.png'>
</center>                    


<center>
    <img src='resources/example_profile_1.png'>
</center>                    


 # <center> Premature optimization is the root of all evil. </center>   
 # <center> Donald Knuth </center>   



## La optimización prematura es la fuente de todos los males.

### <center> Resources: https://github.com/emimartin26/profiling_optimization_pyconar2021 </center>   

 - https://www.webucator.com/blog/2015/08/python-clocks-explained/
 - https://docs.python.org/3/library/time.html
 - https://docs.python.org/es/3/library/timeit.html
 - https://ipython.org/ipython-doc/dev/interactive/magics.html#magic-timeit
 - https://docs.python.org/es/3.10/library/profile.html
 - https://github.com/yousong/yappi
 - https://readthedocs.org/projects/greenletprofiler/
 - https://jiffyclub.github.io/snakeviz/
 - https://github.com/rkern/line_profiler
 - https://docs.python.org/es/3/library/tracemalloc.html
 - https://github.com/pythonprofilers/memory_profiler
 - https://docs.newrelic.com/docs/apm/apm-ui-pages/events/thread-profiler-tool
 - https://docs.datadoghq.com/tracing/profiler/
 - https://www.linkedin.com/learning/faster-python-code

# <center> Thanks! </center>